**What I want to achieve: ** python script for a) creating sqlite tables based on DDL, and b) importing arbitrary csv files into respective tables that match the table structure

## Admin stuff

In [1]:
import sqlite3
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
sns.set(style="darkgrid")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Display any number of columns

from datetime import datetime

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%reload_ext autoreload

C:\Users\M042266\.conda\envs\nips_papers_analysis\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# python-dotenv to load up all the entries in this file as environment variables so they are accessible with os.environ.get.
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# find root directory path
project_root_path = os.path.dirname(find_dotenv())

In [3]:
# creating a path for DATA and SCRIPTS directory
data_dir = os.path.join(project_root_path, 'DATA')
scripts_dir = os.path.join(project_root_path, 'SCRIPTS')

## Connect to database

In [13]:
def execute_sqlite_command(dbpath, scriptpath):
    '''
    Purpose: Runs SQL scripts against the sqlite database but reads no values back. 
             You can use it for create, insert, update and delete statements
    
    Arguments: 
    1) dbpath - the path should include path location including filename
    2) scriptpath - the path should include path location including filename
    
    To do:  Exceptions handling. The current exception handling does nothing
            except tell you “something” went wrong.
    '''
    try:
        # sql connection
        cnx = sqlite3.connect(dbpath)
        cur = cnx.cursor()
       
        # open and close scriptfile after the script has been read
        scriptFile = open(scriptpath, 'r')
        script = scriptFile.read()
        scriptFile.close()
    
        # executescript method executes every line of the SQL script separated by ';'
        # as well as hits commit after everything is done.
        cur.executescript(script)
        #cur.executescript('drop table if exists paper_authors;')   
        
        # hit commit if everything has gone to satisfaction
        cnx.commit()

    except Exception:
        print ("Something went wrong:%s",Exception)


    finally:    
        cnx.close()

In [14]:
execute_sqlite_command(dbpath = os.path.join(data_dir, 'database.sqlite'),
                       scriptpath = os.path.join(scripts_dir,'import.sql'))

## Import CSVs into sqlite tables

In [6]:
df_authors = pd.read_csv(os.path.join(data_dir,"authors.csv"))

In [10]:
df_authors.head()

,id,name
0,1,Hisashi Suzuki
1,10,David Brady
2,100,Santosh S. Venkatesh
3,1000,Charles Fefferman
4,1001,Scott Markel


In [11]:
cnx = sqlite3.connect(os.path.join(data_dir,"database.sqlite"))

In [15]:
pd.read_sql("Select * from authors;", con=cnx)

,id,name


In [17]:
df_authors.to_sql(name= 'authors', con= cnx, if_exists='append', index= False)

In [18]:
pd.read_sql("Select * from authors limit 10;", con=cnx)

,id,name
0,1,Hisashi Suzuki
1,2,Suguru Arimoto
2,3,Philip A. Chou
3,4,John C. Platt
4,5,Alan H. Barr
5,6,Ralph Linsker
6,7,Gene A. Tagliarini
7,8,Edward W. Page
8,9,Ken Hsu
9,10,David Brady
